In [4]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
Mounted at /content/gdrive


In [5]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [12]:
bear_types = 'grizzly','black','teddy'
path = '/content/drive/MyDrive/bears'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
fns = get_image_files(path)
fns

(#417) [Path('/content/drive/MyDrive/bears/teddy/a84b4b0a-adcc-4bb9-9850-206cbdd87728.jpg'),Path('/content/drive/MyDrive/bears/teddy/4df2c2d9-d1ca-41e5-94fc-e565f0ce9daa.jpg'),Path('/content/drive/MyDrive/bears/teddy/03d90b00-91c2-4a66-88bc-78640dd4d0b6.jpg'),Path('/content/drive/MyDrive/bears/teddy/9eece142-8ec7-4d2e-ae65-cb5e4aa2a573.jpeg'),Path('/content/drive/MyDrive/bears/teddy/f7312cc5-32a9-4da4-b7c8-61db83f72b2b.jpeg'),Path('/content/drive/MyDrive/bears/teddy/917d912d-dde3-46aa-b102-faf0ec632ee7.jpeg'),Path('/content/drive/MyDrive/bears/teddy/8f828b01-3242-484a-b583-886914c243d7.jpg'),Path('/content/drive/MyDrive/bears/teddy/7494d54e-b744-40d0-b33a-8abaedeb700e.jpeg'),Path('/content/drive/MyDrive/bears/teddy/33fc0636-28a1-4980-930b-0b508fbadb41.jpeg'),Path('/content/drive/MyDrive/bears/teddy/794dd9cf-8647-49f5-b8f7-7c787ec40a82.jpg')...]

In [15]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [16]:
dls = bears.dataloaders(path)

In [17]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [22]:
learn_inf = load_learner('/content/export.pkl')

In [23]:
#hide_output
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [24]:
#hide
# For the book, we can't actually click an upload button, so we fake it
btn_upload = SimpleNamespace(data = ['images/grizzly.jpg'])

In [25]:
img = PILImage.create(btn_upload.data[-1])

In [26]:
#hide_output
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [27]:
pred,pred_idx,probs = learn_inf.predict(img)

In [28]:
#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: grizzly; Probability: 0.9968')

In [29]:
#hide_output
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [30]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [31]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [32]:
#hide_output
VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])

In [33]:
#hide
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila